## Predicting pIC50 for JAK2 with RNNs and SMILES strings 

This notebook demonstrates how to build predictive recurrent neural network for SMILES strings. We will build classification model for logP with OpenChem Toolkit (https://github.com/Mariewelt/OpenChem)

In [1]:
# Cloning OpenChem. Comment this line if you already cloned the repository
 ! git clone https://github.com/Mariewelt/OpenChem.git

## Imports

In [2]:
import sys
import os

In [3]:
sys.path.append('./OpenChem')

In [4]:
from openchem.models.Smiles2Label import Smiles2Label
from openchem.modules.encoders.rnn_encoder import RNNEncoder
from openchem.modules.mlp.openchem_mlp import OpenChemMLP
from openchem.data.smiles_data_layer import SmilesDataset
from openchem.data.utils import save_smiles_property_file
from openchem.data.utils import create_loader
from openchem.models.openchem_model import build_training, fit, evaluate

In [5]:
import torch.nn as nn
from torch.optim import RMSprop, SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [6]:
from sklearn.metrics import roc_auc_score
import pandas as pd
import copy
import pickle

## Reading data

In [7]:
from openchem.data.utils import read_smiles_property_file
data = read_smiles_property_file('./data/jak2_data.csv', 
                                 cols_to_read=[0, 1])
smiles = data[0]
labels = data[1].astype('float32')

In [8]:
import numpy as np
binary_labels = (labels >= 7.0).astype('float32').reshape(-1)

In [9]:
from openchem.data.utils import get_tokens
tokens, _, _ = get_tokens(smiles)
tokens = tokens + ' '

## Model architecture

Here we define the architecture of our Recurrent Neural Network (RNN). We will use 2 LSTM layers. For more details on how to build models with OpenChem, visit: https://mariewelt.github.io/OpenChem/

In [10]:
import torch
from openchem.utils.utils import identity
from openchem.modules.embeddings.basic_embedding import Embedding
model_object = Smiles2Label

model_params = {
    'use_cuda': True,
    'random_seed': 42,
    'world_size': 1,
    'task': 'classification',
    'data_layer': SmilesDataset,
    'use_clip_grad': False,
    'batch_size': 128,
    'num_epochs': 51,
    'logdir': './logs/jak2_logs',
    'print_every': 1,
    'save_every': 5,
    #'train_data_layer': train_dataset,
    #'val_data_layer': test_dataset,
    'eval_metrics': roc_auc_score,
    'criterion': nn.CrossEntropyLoss(),
    'optimizer': Adam,
    'optimizer_params': {
        'lr': 0.005,
    },
    'lr_scheduler': StepLR,
    'lr_scheduler_params': {
        'step_size': 15,
        'gamma': 0.8
    },
    'embedding': Embedding,
    'embedding_params': {
        'num_embeddings': len(tokens),
        'embedding_dim': 128,
        'padding_idx': tokens.index(' ')
    },
    'encoder': RNNEncoder,
    'encoder_params': {
        'input_size': 128,
        'layer': "LSTM",
        'encoder_dim': 128,
        'n_layers': 2,
        'dropout': 0.8,
        'is_bidirectional': False
    },
    'mlp': OpenChemMLP,
    'mlp_params': {
        'input_size': 128,
        'n_layers': 2,
        'hidden_size': [128, 2],
        'activation': F.relu,
        'dropout': 0.0
    }
}

In [11]:
try:
    os.stat(model_params['logdir'])
except:
    os.mkdir(model_params['logdir'])

## Initializing data splitter for cross validation

In [12]:
from sklearn.model_selection import KFold
cross_validation_split = KFold(n_splits=5, shuffle=True)

In [16]:
data = cross_validation_split.split(smiles, binary_labels)

## Training cross-validated models

In [17]:
import os
i = 0
models = []
results = []
for split in data:
    print('Cross validation, fold number ' + str(i) + ' in progress...')
    train, test = split
    X_train = smiles[train]
    y_train = binary_labels[train].reshape(-1)
    X_test = smiles[test]
    y_test = binary_labels[test].reshape(-1)
    save_smiles_property_file('./data/jak2_train_fold' + str(i) + '.smi', 
                              X_train, y_train.reshape(-1, 1))
    save_smiles_property_file('./data/jak2_test_fold' + str(i) + '.smi', 
                              X_test, y_test.reshape(-1, 1))

    train_dataset = SmilesDataset('./data/jak2_train_fold' + str(i) + '.smi',
                           delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    train_dataset.target = train_dataset.target.reshape(-1)
    test_dataset = SmilesDataset('./data/jak2_test_fold' + str(i) + '.smi',
                       delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    test_dataset.target = test_dataset.target.reshape(-1)
    model_params['train_data_layer'] = train_dataset
    model_params['val_data_layer'] = test_dataset
    model_params['logdir'] = './logs/logp_logs/fold' + str(i)  
    logdir = model_params['logdir']
    ckpt_dir = logdir + '/checkpoint/'
    try:
        os.stat(ckpt_dir)
    except:
        os.mkdir(logdir)
        os.mkdir(ckpt_dir)
    train_loader = create_loader(train_dataset,
                             batch_size=model_params['batch_size'],
                             shuffle=True,
                             num_workers=4,
                             pin_memory=True,
                             sampler=None)
    val_loader = create_loader(test_dataset,
                           batch_size=model_params['batch_size'],
                           shuffle=False,
                           num_workers=1,
                           pin_memory=True)
    models.append(model_object(params=model_params).cuda())
    criterion, optimizer, lr_scheduler = build_training(models[i], model_params)
    results.append(fit(models[i], lr_scheduler, train_loader, optimizer, criterion,
        model_params, eval=True, val_loader=val_loader))
    
    i = i+1

Cross validation, fold number 0 in progress...
TRAINING: [Time: 0m 0s, Epoch: 0, Progress: 0%, Loss: 0.6728]
EVALUATION: [Time: 0m 0s, Loss: 0.6713, Metrics: 0.5521]
TRAINING: [Time: 0m 1s, Epoch: 1, Progress: 1%, Loss: 0.6614]
EVALUATION: [Time: 0m 0s, Loss: 0.6674, Metrics: 0.5778]
TRAINING: [Time: 0m 2s, Epoch: 2, Progress: 3%, Loss: 0.6527]
EVALUATION: [Time: 0m 0s, Loss: 0.6593, Metrics: 0.5662]
TRAINING: [Time: 0m 2s, Epoch: 3, Progress: 5%, Loss: 0.6503]
EVALUATION: [Time: 0m 0s, Loss: 0.6544, Metrics: 0.5874]
TRAINING: [Time: 0m 3s, Epoch: 4, Progress: 7%, Loss: 0.6375]
EVALUATION: [Time: 0m 0s, Loss: 0.6602, Metrics: 0.5826]
TRAINING: [Time: 0m 4s, Epoch: 5, Progress: 9%, Loss: 0.6418]
EVALUATION: [Time: 0m 0s, Loss: 0.6411, Metrics: 0.6157]
TRAINING: [Time: 0m 4s, Epoch: 6, Progress: 11%, Loss: 0.6346]
EVALUATION: [Time: 0m 0s, Loss: 0.6590, Metrics: 0.5770]
TRAINING: [Time: 0m 5s, Epoch: 7, Progress: 13%, Loss: 0.6172]
EVALUATION: [Time: 0m 0s, Loss: 0.6057, Metrics: 0.6477]

TRAINING: [Time: 0m 11s, Epoch: 16, Progress: 31%, Loss: 0.5637]
EVALUATION: [Time: 0m 0s, Loss: 0.5841, Metrics: 0.7067]
TRAINING: [Time: 0m 12s, Epoch: 17, Progress: 33%, Loss: 0.5600]
EVALUATION: [Time: 0m 0s, Loss: 0.5753, Metrics: 0.6949]
TRAINING: [Time: 0m 13s, Epoch: 18, Progress: 35%, Loss: 0.5588]
EVALUATION: [Time: 0m 0s, Loss: 0.5747, Metrics: 0.7056]
TRAINING: [Time: 0m 14s, Epoch: 19, Progress: 37%, Loss: 0.5536]
EVALUATION: [Time: 0m 0s, Loss: 0.5822, Metrics: 0.7035]
TRAINING: [Time: 0m 14s, Epoch: 20, Progress: 39%, Loss: 0.5473]
EVALUATION: [Time: 0m 0s, Loss: 0.5781, Metrics: 0.7076]
TRAINING: [Time: 0m 15s, Epoch: 21, Progress: 41%, Loss: 0.5557]
EVALUATION: [Time: 0m 0s, Loss: 0.5845, Metrics: 0.6969]
TRAINING: [Time: 0m 16s, Epoch: 22, Progress: 43%, Loss: 0.5479]
EVALUATION: [Time: 0m 0s, Loss: 0.5909, Metrics: 0.6872]
TRAINING: [Time: 0m 16s, Epoch: 23, Progress: 45%, Loss: 0.5478]
EVALUATION: [Time: 0m 0s, Loss: 0.6131, Metrics: 0.6321]
TRAINING: [Time: 0m 17s,

EVALUATION: [Time: 0m 0s, Loss: 0.4851, Metrics: 0.7692]
TRAINING: [Time: 0m 23s, Epoch: 33, Progress: 64%, Loss: 0.3928]
EVALUATION: [Time: 0m 0s, Loss: 0.5138, Metrics: 0.7673]
TRAINING: [Time: 0m 23s, Epoch: 34, Progress: 66%, Loss: 0.3835]
EVALUATION: [Time: 0m 0s, Loss: 0.5250, Metrics: 0.7687]
TRAINING: [Time: 0m 24s, Epoch: 35, Progress: 68%, Loss: 0.3777]
EVALUATION: [Time: 0m 0s, Loss: 0.5255, Metrics: 0.7737]
TRAINING: [Time: 0m 25s, Epoch: 36, Progress: 70%, Loss: 0.3905]
EVALUATION: [Time: 0m 0s, Loss: 0.5401, Metrics: 0.7467]
TRAINING: [Time: 0m 25s, Epoch: 37, Progress: 72%, Loss: 0.3805]
EVALUATION: [Time: 0m 0s, Loss: 0.4828, Metrics: 0.7606]
TRAINING: [Time: 0m 26s, Epoch: 38, Progress: 74%, Loss: 0.3687]
EVALUATION: [Time: 0m 0s, Loss: 0.6310, Metrics: 0.7479]
TRAINING: [Time: 0m 27s, Epoch: 39, Progress: 76%, Loss: 0.3671]
EVALUATION: [Time: 0m 0s, Loss: 0.5460, Metrics: 0.7511]
TRAINING: [Time: 0m 27s, Epoch: 40, Progress: 78%, Loss: 0.3820]
EVALUATION: [Time: 0m 0s

TRAINING: [Time: 0m 35s, Epoch: 49, Progress: 96%, Loss: 0.4818]
EVALUATION: [Time: 0m 0s, Loss: 0.5364, Metrics: 0.7615]
TRAINING: [Time: 0m 35s, Epoch: 50, Progress: 98%, Loss: 0.4711]
EVALUATION: [Time: 0m 0s, Loss: 0.5435, Metrics: 0.7549]
Cross validation, fold number 4 in progress...
TRAINING: [Time: 0m 0s, Epoch: 0, Progress: 0%, Loss: 0.6746]
EVALUATION: [Time: 0m 0s, Loss: 0.6490, Metrics: 0.5852]
TRAINING: [Time: 0m 1s, Epoch: 1, Progress: 1%, Loss: 0.6624]
EVALUATION: [Time: 0m 0s, Loss: 0.6315, Metrics: 0.5787]
TRAINING: [Time: 0m 2s, Epoch: 2, Progress: 3%, Loss: 0.6444]
EVALUATION: [Time: 0m 0s, Loss: 0.6750, Metrics: 0.5824]
TRAINING: [Time: 0m 2s, Epoch: 3, Progress: 5%, Loss: 0.6289]
EVALUATION: [Time: 0m 0s, Loss: 0.6274, Metrics: 0.6320]
TRAINING: [Time: 0m 3s, Epoch: 4, Progress: 7%, Loss: 0.6142]
EVALUATION: [Time: 0m 0s, Loss: 0.6460, Metrics: 0.6130]
TRAINING: [Time: 0m 4s, Epoch: 5, Progress: 9%, Loss: 0.5972]
EVALUATION: [Time: 0m 0s, Loss: 0.5814, Metrics: 0.6

## Evaluating the models

In [18]:
import numpy as np
rmse = []
auc_score = []
for i in range(5):
    test_dataset = SmilesDataset('./data/jak2_test_fold' + str(i) + '.smi',
                                 delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    test_dataset.target = test_dataset.target.reshape(-1)
    val_loader = create_loader(test_dataset,
                               batch_size=model_params['batch_size'],
                               shuffle=False,
                               num_workers=1,
                               pin_memory=True)
    metrics = evaluate(models[i], val_loader, criterion)
    rmse.append(np.sqrt(metrics[0]))
    auc_score.append(metrics[1])

EVALUATION: [Time: 0m 0s, Loss: 0.5376, Metrics: 0.7529]
EVALUATION: [Time: 0m 0s, Loss: 0.5504, Metrics: 0.7556]
EVALUATION: [Time: 0m 0s, Loss: 0.5522, Metrics: 0.7740]
EVALUATION: [Time: 0m 0s, Loss: 0.5435, Metrics: 0.7549]
EVALUATION: [Time: 0m 0s, Loss: 0.5699, Metrics: 0.7641]


In [19]:
print("Cross-validated RMSE: ",  np.mean(rmse))
print("Cross-validated AUC score: ", np.mean(auc_score))

Cross-validated RMSE:  0.742072568424805
Cross-validated AUC score:  0.7603019075230668
